In [1]:
import itertools

import awkward as ak
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from rich import print
from tabulate import tabulate

from apps.prod4a_merge_study import BestCut, EventSelection, PFOSelection, SignalBackground, SplitSample, ShowerMergeQuantities
from python.analysis import Master, Plots, vector
plt.style.use('dark_background')

In [2]:
events = Master.Data("work/ROOTFiles/Prod4a_6GeV_BeamSim_00.root", True)#, nEvents=1000)
EventSelection(events)
PFOSelection(events)
start_showers, to_merge = SplitSample(events)
signal, background, signal_all = SignalBackground(events, start_showers, to_merge)

/home/sb16165/Documents/pi0-analysis/analysis/python/analysis/Master.py:52: UserWarning: Couldn't apply filters to filters.
  warnings.warn(f"Couldn't apply filters to {var}.")
/home/sb16165/Documents/pi0-analysis/analysis/python/analysis/Master.py:52: UserWarning: Couldn't apply filters to filename.
  warnings.warn(f"Couldn't apply filters to {var}.")
/home/sb16165/Documents/pi0-analysis/analysis/python/analysis/Master.py:52: UserWarning: Couldn't apply filters to _TrueParticleData__pi0_MC.
  warnings.warn(f"Couldn't apply filters to {var}.")


'BeamMCFilter' executed in 14.3445s
number of dalitz decays: 222
'ApplyBeamFilter' executed in 8.2128s


\begin{tabular}{lllll}
\hline
 event selection                   & type        & number of events & percentage of events removed & percentage of 
events remaining \\
 no selection                      & -           & 100222           & -                            &               
\\
 beam -\ensuremath{>} pi0 + X                   & truth       & 27341            & 72.71956257109217            & 
100                            \\
 pi+ beam                          & backtracked & 19949            & 27.036319081233312           & 
72.96368091876668              \\
 diphoton decay                    & truth       & 19727            & 1.1128377362273798           & 
72.15171354376211              \\
 beam particle                     & reco        & 10017            & 49.22187864348355            & 
36.637284664057646             \\
 nPFP \ensuremath{>} 1                          & reco        & 10017            & 0.0                          & 
36.637284664057646             \\
 at least 1 true particle          & backtracked & 10017            & 0.0                          & 
36.637284664057646             \\
 both true photons are backtracked & backtracked & 4356             & 56.51392632524708            & 
15.932116601441052             \\
\hline
\end{tabular}

'EventSelection' executed in 133.9671s


Total number of PFOs: 2645612

...

False

True

True

True

'SplitSample' executed in 37.1865s


In [7]:
def CheckSignalFromParticleTree(events, eventNum):
    number = events.recoParticles.number[eventNum]
    mother = events.recoParticles.mother[eventNum]

    def FindMyChildren(parents):
        tree = []
        for i in range(ak.count(number[parents])):
            children = mother == number[parents][i]
            nchildren = ak.count(ak.ravel(children[children]))

            if ak.count(tree) == 0:
                tree = children
            else:
                tree = tree & children

            if nchildren > 0:
                # we need to go deeper!
                #tree.append(FindMyChildren(children))
                tree = tree & FindMyChildren(children)
            else:
                # no children :(
                pass
        return tree

    parent_particles = mother == -999
    start_showers_all = np.logical_or(*start_showers)[eventNum]

    family_tree = FindMyChildren(start_showers_all)
    #print(family_tree)

    signal_in_child = []
    for i in range(len(family_tree)):
        if type(family_tree[i]) == list:
            continue
        else:
            signal_in_child.append(signal_all[eventNum][family_tree[i]])
    return ak.Array(signal_in_child)
output = []
for i in range(ak.count(events.eventNum)):
    output.append(CheckSignalFromParticleTree(events, i))

In [ ]:
print(output)

If there are no bugs this concludes that most start showers don't have a daughter and that the daughter is not usually a signal particle (can it still be the case?)

In [12]:
# try the opposite, find signal and then check their mothers

start_shower_number = [events.recoParticles.number[start_showers[i]] for i in range(2)]
signal_mother = events.recoParticles.mother[signal_all]

print(start_shower_number)
print(signal_mother)


signal_children = [ak.any(signal_mother == ak.flatten(start_shower_number[i]), 1) for i in range(2)]
print(signal_children) # if false then no children of the start showers are signal PFOs

signal_children = np.logical_or(*signal_children)
print(f"number of events where a signal PFO is a child of the starting shower: {ak.count(signal_children[signal_children])}")
# TODO: run on more events 

[
    <Array [[484], [5], [132], ... 56], [424], [1]] type='4356 * option[var * int32]'>,
    <Array [[0], [59], [344], ... [1], [78], [455]] type='4356 * option[var * int32]'>
]

[[-999, 9, -999], [-999], [9], ... -999, 5, -999, 7, 4, -999, 10, -999, -999, -999]]

[
    <Array [False, False, False, ... False, False] type='4356 * ?bool'>,
    <Array [False, False, False, ... True, False] type='4356 * ?bool'>
]

number of events where a signal PFO is a child of the starting shower: 39